In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
file_path = "../data/raw/data_all.xltx"

df = pd.read_excel(file_path, sheet_name="data")
df.head()


,account_id,SourceSystem,activity_date,who_id,opportunity_id,opportunity_stage,is_lead,types,Country,solution
0,0010L00001hVmFhQAK,SFDC_US,2022-07-25,0030L00001vlbHLQAY,NaN,no_opp,1.0,Follow Up,US,MRS
1,0010L00001hVmFhQAK,SFDC_US,2023-02-08,0034X00002xZlQtQAK,NaN,no_opp,1.0,Inbound Call,US,MRS
2,0010L00001hVmFhQAK,SFDC_US,2023-02-14,0030L00001vlbHLQAY,NaN,no_opp,1.0,Inbound Call,US,MRS
3,0010L00001hVmFhQAK,SFDC_US,2023-02-20,0030L00001vlbHLQAY,NaN,no_opp,2.0,Inbound Call,US,MRS
4,0010L00001hVmFhQAK,SFDC_US,2023-03-16,0034X00003GOUrFQAX,NaN,no_opp,1.0,Inbound Call,US,MRS


In [3]:
df.info()
df.isna().sum()
df.nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259917 entries, 0 to 259916
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   account_id         259917 non-null  object        
 1   SourceSystem       259917 non-null  object        
 2   activity_date      205516 non-null  datetime64[ns]
 3   who_id             238294 non-null  object        
 4   opportunity_id     45849 non-null   object        
 5   opportunity_stage  259901 non-null  object        
 6   is_lead            259901 non-null  float64       
 7   types              259901 non-null  object        
 8   Country            259730 non-null  object        
 9   solution           259901 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 19.8+ MB


account_id           13293
SourceSystem             6
activity_date         4718
who_id               56849
opportunity_id        8003
opportunity_stage       24
is_lead                  2
types                   12
Country                 54
solution                 3
dtype: int64

In [4]:
for col in df.select_dtypes(include="object"):
    df[col] = df[col].str.strip()


In [5]:
df["activity_date"] = pd.to_datetime(df["activity_date"], errors="coerce")

df = df[df["activity_date"].notna()]
df = df[
    (df["activity_date"].dt.year >= 2005) &
    (df["activity_date"].dt.year <= dt.date.today().year + 1)
]


In [ ]:
df["types"] = df["types"].replace({
    "Inbound Call": "Call",
    "Outbound Call": "Call"
})
df["types"] = df["types"].str.title()
df["types"] = df["types"].fillna("Unknown").astype(str).str.strip()


In [ ]:
df["Country"] = df["Country"].fillna("UNKNOWN")
df["solution"] = df["solution"].fillna(df["solution"].mode()[0])
df["is_lead"] = df["is_lead"].fillna(df["is_lead"].mode()[0]).astype(int)


In [ ]:
dedup_key = ["account_id", "activity_date", "types", "who_id", "opportunity_id"]

df = df.sort_values(dedup_key)
df = df.drop_duplicates(subset=dedup_key)


In [ ]:
df["activity_order"] = df.groupby("account_id")["activity_date"].rank(method="first")
df["action_weight"] = 1.0


In [ ]:
df.to_csv("../data/processed/cleaned_activities.csv", index=False)


In [ ]:
df.head()
df.shape
df.isna().sum()